### I. 元类

### III. 并发编程

#### 协程 - coroutine 
绿色线程，轻量级的线程。

* 示例1，用coroutine代替线程

In [7]:
def countdown(n):
    while n > 0:
        print('T-minus', n)
        yield
        n -= 1
    print('Blastoff!')
    
    
def countup(n):
    x = 0
    while x < n:
        print('Conuting up', x)
        yield
        x += 1

In [4]:
from collections import deque

class TaskScheduler:
    
    def __init__(self):
        self._task_queue = deque()
        
    def new_task(self, task):
        '''Admit a newly started task to the scheduler'''
        self._task_queue.append(task)
    
    def run(self):
        while self._task_queue:
            task = self._task_queue.popleft()
            
            try:
                next(task)
                self._task_queue.append(task) # 轮询的运行queue里的协程
            except StopIteration:
                pass

In [12]:
scheduler = TaskScheduler()
scheduler.new_task(countdown(5))
scheduler.new_task(countup(6))
scheduler.run()

T-minus 5
Conuting up 0
T-minus 4
Conuting up 1
T-minus 3
Conuting up 2
T-minus 2
Conuting up 3
T-minus 1
Conuting up 4
Blastoff!
Conuting up 5


* 示例2

In [22]:
class ActorScheduler:
    
    def __init__(self):
        self._actors = {}
        self._msg_queue = deque()
        
    def new_actor(self, name, actor):
        self._msg_queue.append((actor, None))
        self._actors[name] = actor
       
    def send(self, name, msg):
        actor = self._actors.get(name)
        if actor:
            self._msg_queue.append((actor, msg))
            
    def run(self):
        while self._msg_queue:
            actor, msg = self._msg_queue.popleft()
            
            try:
                actor.send(msg)
                print("Sending ", msg, " to ", actor.__name__)
            except StopIteration:
                pass

In [23]:
def printer():
    while True:
        msg = yield
        print('Got: ', msg)

def counter(sched):
    
    while True:
        n = yield
        if n == 0:
            break
        
        sched.send('printer', n)
        
        sched.send('counter', n-1)

In [24]:
sched = ActorScheduler()
sched.new_actor('printer', printer())
sched.new_actor('counter', counter(sched))

sched.send('counter', 3)
sched.run()

Sending  None  to  printer
Sending  None  to  counter
Sending  3  to  counter
Got:  3
Sending  3  to  printer
Sending  2  to  counter
Got:  2
Sending  2  to  printer
Sending  1  to  counter
Got:  1
Sending  1  to  printer


* 3

In [1]:
from select import select
from collections import deque

class YieldEvent:
    def handle_yield(self, sched, task):
        pass
    
    def handle_resume(self, sched, task):
        pass

    
class Scheduler:
    
    def __init__(self):
        self._numtasks = 0
        self._ready = deque()
        self._read_waiting = {}
        self._write_waiting = {}
    
    def _iopoll(self):
        rset, wset, eset = select(self._read_waiting, 
                                 self._write_waiting, [])
        
        for r in rset:
            evt, task = self._read_waiting.pop(r)
            evt.handle_resume(self, task)
        
        for w in wset:
            evt, task = self._write_waiting.pop(w)
            evt.handle_resume(self, task)
    
    def new(self, task):
        self._ready.append((task, None))
        self._numtasks += 1
    
    def add_ready(self, task, msg=None):
        self._ready.append((task, msg))
        
    def _read_wait(self, fileno, evt, task):
        self._read_waiting[fileno] = (evt, task)
    
    def _write_wait(self, fileno, evt, task):
        self._write_waiting[fileno] = (evt, task)
    
    def run(self):    
        while self._numtasks:
        
            if not self._ready:
                self._iopoll()
            task, msg = self._ready.popleft()
            
            try:
                r = task.send(msg)
                if isinstance(r, YieldEvent):
                    r.handle_yield(self, task)
                else:
                    raise RuntimeError('Unrecognized yield event')
            except StopIteration:
                self._numtasks -= 1

In [2]:
class ReadSocket(YieldEvent):
    
    def __init__(self, sock, nbytes):
        self.sock = sock
        self.nbytes = nbytes
    
    def handle_yield(self, sched, task):
        sched._read_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        data = self.sock.recv(self.nbytes)
        print("recieved data ", data)
        sched.add_ready(task, data)
    
class WriteSocket(YieldEvent):
    
    def __init__(self, sock, data):
        self.sock = sock
        self.data = data
    
    def handle_yield(self, sched, task):
        sched._write_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        nsent = self.sock.send(self.data)
        sched.add_ready(task, nsent)
    
class AcceptSocket(YieldEvent):
    
    def __init__(self, sock):
        self.sock = sock
    
    def handle_yield(self, sched, task):
        sched._read_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        r = self.sock.accept()
        sched.add_ready(task, r)
    
class Socket(object):
    
    def __init__(self, sock):
        self._sock = sock
    
    def recv(self, maxbytes):
        return ReadSocket(self._sock, maxbytes)
    
    def send(self, data):
        return WriteSocket(self._sock, data)
    
    def accept(self):
        return AcceptSocket(self._sock)
    
    def __getattr__(self, name):
        return getattr(self._sock, name)

In [3]:
from socket import socket, AF_INET, SOCK_STREAM
import time


def readline(sock):
    chars = []
    while True:
        c = yield sock.recv(1)
        if not c:
            break
        
        chars.append(c)
        if c == b'\n':
            break
    return b''.join(chars)


class EchoServer:
    
    def __init__(self, addr, sched):
        self.sched = sched
        sched.new(self.server_loop(addr))
    
    def server_loop(self, addr):
        s = Socket(socket(AF_INET, SOCK_STREAM))
        
        s.bind(addr)
        s.listen(5)
        
        while True:
            c, a = yield s.accept()
            print('Got connnection from ', a)
            self.sched.new(self.client_handler(Socket(c)))
        
    def client_handler(self, client):
        while True:
            line = yield from readline(client)
            if not line:
                break
            
            line = b'Got: ' + line
            while line:
                nsent = yield client.send(line)
                line = line[nsent:]
            
            client.close()
            print('Client closed')

In [4]:
sched = Scheduler()
EchoServer(('', 1600), sched)
sched.run()

Got connnection from  ('127.0.0.1', 54816)
recieved data  b'H'
recieved data  b'e'
recieved data  b'l'
recieved data  b'l'
recieved data  b'o'
recieved data  b''
Client closed


ValueError: file descriptor cannot be a negative integer (-1)

In [5]:
import time
dir(time.time)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

In [6]:
from datetime import datetime

In [7]:
dir(datetime)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'astimezone',
 'combine',
 'ctime',
 'date',
 'day',
 'dst',
 'fromordinal',
 'fromtimestamp',
 'hour',
 'isocalendar',
 'isoformat',
 'isoweekday',
 'max',
 'microsecond',
 'min',
 'minute',
 'month',
 'now',
 'replace',
 'resolution',
 'second',
 'strftime',
 'strptime',
 'time',
 'timestamp',
 'timetuple',
 'timetz',
 'today',
 'toordinal',
 'tzinfo',
 'tzname',
 'utcfromtimestamp',
 'utcnow',
 'utcoffset',
 'utctimetuple',
 'weekday',
 'year']

In [8]:
from time import time

In [11]:
datetime.time(1558658909053)

TypeError: descriptor 'time' requires a 'datetime.datetime' object but received a 'int'

In [13]:
s = 1558658909053
datetime.fromtimestamp()

TypeError: Required argument 'timestamp' (pos 1) not found

In [16]:
import time
dir(time)

['_STRUCT_TM_ITEMS',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'altzone',
 'asctime',
 'clock',
 'ctime',
 'daylight',
 'get_clock_info',
 'gmtime',
 'localtime',
 'mktime',
 'monotonic',
 'perf_counter',
 'process_time',
 'sleep',
 'strftime',
 'strptime',
 'struct_time',
 'time',
 'timezone',
 'tzname']

In [18]:
time.gmtime(s)

OSError: [Errno 22] Invalid argument